In [1]:
import os
import torch

import os
import sys

import numpy as np
import pandas as pd

import torch
import torch.nn as nn


sys.path.append('..')
sys.path.append('.')
sys.path.append('../llm_garden')

from peft_modules import peft_utils
import esm_adapter



DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# torch.hub.set_dir("./torch_hub")
torch.hub.set_dir("/home/zengs/zengs_data/torch_hub")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dtype = torch.double



SMOKE_TEST = os.environ.get("SMOKE_TEST")
N_TRIALS = 3 if not SMOKE_TEST else 2
N_BATCH = 20 if not SMOKE_TEST else 2
MC_SAMPLES = 256 if not SMOKE_TEST else 32

/home/zengs/data/anaconda3/envs/venv_pl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
INPUT_SEQ_SIZE = 10


# data
data = peft_utils.get_esm_example_data(INPUT_SEQ_SIZE)


# Load ESM-2 model
# esm2_t33_650M_UR50D
# esm2_t6_8M_UR50D
model, alphabet = esm_adapter.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()

batch_labels, batch_strs, batch_tokens = batch_converter(data)

batch_tokens, batch_tokens.shape

(tensor([[ 0,  5, 15, 11,  7, 10, 16,  9, 10,  4, 15,  2],
         [ 0, 25,  5,  4, 11,  5, 10, 16, 16,  9,  7,  2],
         [ 0, 13, 15,  9,  7, 18, 13,  4, 12, 10, 13,  2],
         [ 0,  9, 10, 13, 21, 12,  8, 16, 11,  6, 20,  2]]),
 torch.Size([4, 12]))

In [3]:
token_embedding = model.embed_tokens(batch_tokens)

# token_embedding (torch.Tensor of shape (4, 12, 320) ((N_SEQ, N_TOKEN, N_FEAT)))
# The sequence of token embeddings for each input sequence in the batch.
# It is a sample data.
token_embedding, token_embedding.shape

(tensor([[[-1.9543e-01, -2.7515e-01, -1.4941e-01,  ...,  1.2744e-01,
           -1.0431e-01, -6.4697e-02],
          [-6.6711e-02, -1.1932e-02,  8.6288e-03,  ...,  1.4633e-02,
           -4.3945e-02, -1.9348e-01],
          [-9.7595e-02,  1.5762e-02,  3.3600e-02,  ...,  3.4027e-03,
            2.2369e-02,  2.8732e-02],
          ...,
          [-8.5693e-02, -2.2476e-02,  4.2603e-02,  ...,  8.2764e-02,
            8.9539e-02,  8.9417e-02],
          [-9.7595e-02,  1.5762e-02,  3.3600e-02,  ...,  3.4027e-03,
            2.2369e-02,  2.8732e-02],
          [ 7.3120e-02, -3.0811e-01, -1.1700e-01,  ..., -2.2058e-01,
           -3.4326e-01, -1.5488e-03]],
 
         [[-1.9543e-01, -2.7515e-01, -1.4941e-01,  ...,  1.2744e-01,
           -1.0431e-01, -6.4697e-02],
          [ 2.0813e-02, -8.6548e-02, -7.0374e-02,  ..., -2.9898e-04,
           -8.3923e-02, -1.3000e-01],
          [-6.6711e-02, -1.1932e-02,  8.6288e-03,  ...,  1.4633e-02,
           -4.3945e-02, -1.9348e-01],
          ...,
    